using:
    
    Riddhi_SLAM/20181008-210159ion_bright_matrix.npz

In [1]:
MAXMSMT = 100

In [2]:
import numpy as np
import matplotlib

fsize=8
Fsize=8
# Set global parameters
matplotlib.rcParams['font.size'] = fsize # global
matplotlib.rcParams['font.family'] = 'sans-serif'
matplotlib.rcParams['font.sans-serif'] = 'Arial'
matplotlib.rcParams['axes.linewidth'] = 0.5
matplotlib.rcParams['mathtext.default'] ='regular' # makes mathtext mode Arial. note mathtext is used as ticklabel font in log plots

# Set global tick mark parameters
matplotlib.rcParams['xtick.major.width'] = 0.5
matplotlib.rcParams['ytick.major.width'] = 0.5
matplotlib.rcParams['xtick.labelsize']= fsize
matplotlib.rcParams['ytick.labelsize'] = fsize
matplotlib.rcParams['xtick.minor.visible'] = False
matplotlib.rcParams['ytick.minor.visible'] = False
matplotlib.rcParams['xtick.direction'] = 'in'
matplotlib.rcParams['ytick.direction'] = 'in'


import sys
sys.path.append('../') # where to find qslam folder
import copy
## import qslam modules

from exptrisk import EmpiricalRisk
from qslamdesignparams import GLOBALDICT 
import matplotlib.pyplot as plt
from visualiserisk import DataCube, cm2inch

/home/riddhisw/Documents/SLAM_project/qslam/expt_data/Riddhi_SLAM/20181008-210159ion_bright_matrix.npz


In [3]:
num_qubits = 5 # number of ions in the trap

In [4]:
change_gridconfig = True

# assume equi-distant linear array
if change_gridconfig is True:
    GLOBALDICT["GRIDDICT"] = {}
    for idx_posy in range(num_qubits):
        if idx_posy < 10 :
            GLOBALDICT["GRIDDICT"]["QUBIT_0" + str(idx_posy)] = (0.0, float(idx_posy))
        if idx_posy >= 10 :
            GLOBALDICT["GRIDDICT"]["QUBIT_" + str(idx_posy)] = (0.0, float(idx_posy))

In [5]:
change_MAX_NUM_ITERATIONS = 5
change_MSMTS_PER_NODE = 1
change_SIGMOID_APPROX_ERROR = 10.0**(-6)
change_QUANTISATION_UNCERTY = 10.0**(-4)
change_P_ALPHA = 15 
change_P_BETA = 10 
change_LAMBDA_1 = 0.820
change_LAMBDA_2 = 0.968

GLOBALDICT["MODELDESIGN"]["MAX_NUM_ITERATIONS"] = 5
GLOBALDICT["MODELDESIGN"]["MSMTS_PER_NODE"] = 1
GLOBALDICT["NOISEPARAMS"]["SIGMOID_APPROX_ERROR"]["SIGMA"] = change_SIGMOID_APPROX_ERROR
GLOBALDICT["NOISEPARAMS"]["QUANTISATION_UNCERTY"]["SIGMA"] = change_QUANTISATION_UNCERTY
GLOBALDICT["MODELDESIGN"]["P_ALPHA"] = change_P_ALPHA
GLOBALDICT["MODELDESIGN"]["P_BETA"] = change_P_BETA
GLOBALDICT["MODELDESIGN"]["LAMBDA_1"] = change_LAMBDA_1
GLOBALDICT["MODELDESIGN"]["LAMBDA_2"] = change_LAMBDA_2

In [6]:
meta_max_iter_scan = [ 5, 10, 15, 20, 25, 50, 75, 100] #, 125, 250]

lambda1 = [0.99, 0.956, 0.922, 0.888, 0.854, 0.820, 0.786, 0.752, 0.718, 0.684, 0.65]
lambda2 = [0.977, 0.9752, 0.9734, 0.9716, 0.9698, 0.968, 0.9662, 0.9644, 0.9626, 0.9608, 0.959]
lambda_scan = zip(lambda1, lambda2)

msmt_per_qubit_scan = [1, 2, 4, 5, 6, 8, 10, 15, 20, 25, 50]


LOOPS_DICT = {"meta_max_iter_scan":meta_max_iter_scan, 
              "lambda_scan":lambda_scan,
              "msmt_per_qubit_scan": msmt_per_qubit_scan}

ParamUpdater = DataCube(LOOPS_DICT)

In [7]:
SAMPLE_GLOBAL_MODEL = copy.deepcopy(GLOBALDICT)

meta_truth = []
meta_qslam = []
meta_naive = []

for idx_msmtpernode in range(len(ParamUpdater.lambda_scan)):
    
    print
    print "Using %s lambda ...", ParamUpdater.lambda_scan[idx_msmtpernode]

    SAMPLE_GLOBAL_MODEL["MODELDESIGN"]["LAMBDA_1"] = ParamUpdater.lambda_scan[idx_msmtpernode][0]
    SAMPLE_GLOBAL_MODEL["MODELDESIGN"]["LAMBDA_2"] = ParamUpdater.lambda_scan[idx_msmtpernode][1]
    
    qslam = []
    truth = []
    naive = []

    
    for idx_msmt_iter in range(len(ParamUpdater.meta_max_iter_scan)):
    
        SAMPLE_GLOBAL_MODEL["MODELDESIGN"]["MAX_NUM_ITERATIONS"] = ParamUpdater.meta_max_iter_scan[idx_msmt_iter]
        
        print "idx_msmt_iter", idx_msmt_iter, SAMPLE_GLOBAL_MODEL["MODELDESIGN"]["MAX_NUM_ITERATIONS"]
        print SAMPLE_GLOBAL_MODEL["MODELDESIGN"]["LAMBDA_1"], SAMPLE_GLOBAL_MODEL["MODELDESIGN"]["LAMBDA_2"]
        
        expt = EmpiricalRisk(SAMPLE_GLOBAL_MODEL)

        qfmap, rmap, path = expt.qslam_trial()
        nfmap, truemap = expt.naive_trial()
        
        truth.append(truemap)
        qslam.append(qfmap)
        naive.append(nfmap)

        print "example path:", path
        print "done"
        print 
        print
    
    
    meta_truth.append(truth)
    meta_qslam.append(qslam)
    meta_naive.append(naive)

    
    np.savez('qslam_Ramseys_pA_v2_lambdascan_MAPS', 
              meta_truth=meta_truth,
              meta_qslam=meta_qslam,
              meta_naive=meta_naive)


Using %s lambda ... (0.99, 0.977)
idx_msmt_iter 0 5
0.99 0.977
example path: [4, 1, 0, 3, 2]
done


idx_msmt_iter 1 10
0.99 0.977
example path: [2, 1, 4, 3, 0, 2, 0, 2, 0, 4]
done


idx_msmt_iter 2 15
0.99 0.977
example path: [0, 4, 1, 2, 3, 4, 2, 4, 2, 4, 3, 4, 3, 4, 3]
done


idx_msmt_iter 3 20
0.99 0.977
example path: [0, 2, 4, 3, 1, 0, 4, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3]
done


idx_msmt_iter 4 25
0.99 0.977
example path: [0, 3, 4, 2, 1, 0, 3, 4, 3, 4, 3, 4, 3, 4, 3, 2, 3, 2, 3, 2, 4, 2, 4, 2, 4]
done


idx_msmt_iter 5 50
0.99 0.977
example path: [3, 1, 2, 0, 4, 2, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 4, 1, 4, 3, 4, 3, 4, 3, 4, 3, 4, 3, 4, 3, 4, 3, 4, 3, 4]
done


idx_msmt_iter 6 75
0.99 0.977
example path: [3, 2, 4, 1, 0, 1, 0, 3, 0, 3, 0, 3, 4, 3, 4, 2, 0, 2, 0, 2, 0, 1, 0, 1, 2, 1, 2, 0, 2, 0, 2, 0, 2, 0, 1, 0, 2, 0, 2, 0, 2, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 4, 0, 4, 1, 4]
done


id

../particlesets.py:162: RuntimeWarning: overflow encountered in double_scalars
  normalisation = 1.0/unnormalised_total
../particlesets.py:163: RuntimeWarning: invalid value encountered in multiply
  return normalisation*raw_weights


example path: [4, 3, 2, 1, 0, 1, 2, 1, 2, 4]
done


idx_msmt_iter 2 15
0.65 0.959
example path: [2, 3, 4, 1, 0, 3, 4, 3, 0, 1, 0, 3, 0, 1, 0]
done


idx_msmt_iter 3 20
0.65 0.959
example path: [1, 4, 3, 0, 2, 3, 0, 4, 1, 2, 1, 4, 2, 4, 2, 4, 2, 4, 2, 4]
done


idx_msmt_iter 4 25
0.65 0.959
example path: [2, 1, 4, 3, 0, 2, 0, 2, 0, 2, 3, 2, 3, 1, 2, 0, 2, 0, 2, 3, 1, 3, 1, 3, 1]
done


idx_msmt_iter 5 50
0.65 0.959
example path: [1, 3, 4, 0, 2, 1, 2, 3, 2, 3, 0, 3, 0, 3, 0, 4, 0, 4, 0, 1, 0, 1, 0, 2, 0, 2, 0, 1, 2, 0, 2, 0, 2, 0, 2, 0, 1, 2, 1, 2, 1, 2, 0, 2, 0, 2, 0, 1, 0, 3]
done


idx_msmt_iter 6 75
0.65 0.959
example path: [4, 1, 3, 2, 0, 1, 2, 0, 2, 1, 3, 0, 3, 1, 3, 2, 4, 3, 2, 1, 2, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0]
done


idx_msmt_iter 7 100
0.65 0.959
example path: [1, 2, 3, 4, 0, 3, 1, 3, 4, 3, 4, 3, 4, 3, 4, 3, 4, 3, 4, 3, 4, 3, 4, 3, 4, 3, 0, 3, 0, 